In [1]:
import numpy as np
np.set_printoptions(precision=1)
import matplotlib.pylab as plt
import torch
from modules.utils import load_cifar10
from modules.cnn_with_spectral_parameterization import CNN_Spectral_Param
from modules.cnn_with_spectral_pooling import CNN_Spectral_Pool
from modules.cnn_all_spectral import Spectral_CNN

import warnings
warnings.filterwarnings('ignore')

% matplotlib inline

In [2]:
# In the interest of training time, we only used 1 of 5 cifar10 batches
# The important part of the experiment is to compare the rates of convergence of training accuracy,
# so subsetting the training dataset for both spectral and spatial models shouldn't impact
# the relationship between their train accuracy convergences
traindata, testdata = load_cifar10(1, channels_last=False)

file already downloaded..
getting batch 1


In [3]:
# Size of training set 
print(len(traindata))

# Size of one training sample 
print(traindata[0][0].size()) 

# Datatype of training set 
print(type(traindata[0][0]))

10000
torch.Size([3, 32, 32])
<class 'torch.FloatTensor'>


# Spectral Pooling Architecture - CPU / Numpy fft

Very slow. 

In [ ]:
pool = CNN_Spectral_Pool(device_type='cpu')
pool.train(traindata, epochs=3)

# Spectral Pooling Architecture - GPU / Pytorch fft 

In [ ]:
pool_cuda = CNN_Spectral_Pool(device_type='gpu')
pool_cuda.train(traindata, epochs=3)

# Spectral Parameterization - Generic Architecture 

In [5]:
generic_spectral = CNN_Spectral_Param(architecture='generic', use_spectral_params=True, device_type='cpu')
generic_spectral.train(traindata, epochs=5) 

Epoch 0 Train Error: 0.9026
Epoch 1 Train Error: 0.8637
Epoch 2 Train Error: 0.8919
Epoch 3 Train Error: 0.8966
Epoch 4 Train Error: 0.8684


In [4]:
generic = CNN_Spectral_Param(architecture='generic', use_spectral_params=False, device_type='cpu') 
generic.train(traindata, epochs=10)

Epoch 0 Train Error: 0.7029
Epoch 1 Train Error: 0.6975
Epoch 2 Train Error: 0.6401
Epoch 3 Train Error: 0.6028
Epoch 4 Train Error: 0.5832
Epoch 5 Train Error: 0.5623
Epoch 6 Train Error: 0.5505
Epoch 7 Train Error: 0.5369
Epoch 8 Train Error: 0.5253
Epoch 9 Train Error: 0.504


# Spectral Parameterization - Deep Architecture 


Kernels will crash on CPU only device. Test on GPU only. 

In [ ]:
deep_spectral = CNN_Spectral_Param(architecture='deep', use_spectral_params=True, device_type='gpu')
deep_spectral.train(traindata, epochs=5)

In [ ]:
deep = CNN_Spectral_Param(architecture='deep', use_spectral_params=False, device_typ='gpu')
deep.train(traindata, epochs=5)

# Spectral Parameterization - Convolution Completely in Spectral Domain 

In the paper the authors initialize filters in the spectral domain, but then only use the real part to do convolution as usual. 

We can also test doing convolution completely in the spectral domain by using pointwise multiplication between a spectral filter and the input converted into the spectral domain. 

In [ ]:
spectral_cnn = Spectral_CNN()
spectral_cnn.train(traindata, epochs=5)